In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import data
import mne
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import os
import scipy
import pingouin as pg
from os import listdir
from os.path import isfile, join

sns.set(font_scale=1)

## Loading Data

In [2]:
path = 'C:/Users/bribeiroriccioppo/Desktop/mental-fatigue/Raw Data/'
xdf_files = [xdf for xdf in listdir(path) if isfile(join(path, xdf))]

for i in range(len(xdf_files)):
    xdf_files[i] = path + xdf_files[i]

In [3]:
xdf_files

['C:/Users/bribeiroriccioppo/Desktop/mental-fatigue/Raw Data/S10_2022-05-24.xdf',
 'C:/Users/bribeiroriccioppo/Desktop/mental-fatigue/Raw Data/S11_2022-05-30.xdf',
 'C:/Users/bribeiroriccioppo/Desktop/mental-fatigue/Raw Data/S12_2022-05-31.xdf',
 'C:/Users/bribeiroriccioppo/Desktop/mental-fatigue/Raw Data/S13_2022-06-01.xdf',
 'C:/Users/bribeiroriccioppo/Desktop/mental-fatigue/Raw Data/S14_2022-06-03.xdf',
 'C:/Users/bribeiroriccioppo/Desktop/mental-fatigue/Raw Data/S15_2022-06-07.xdf',
 'C:/Users/bribeiroriccioppo/Desktop/mental-fatigue/Raw Data/S16_2022-06-08.xdf',
 'C:/Users/bribeiroriccioppo/Desktop/mental-fatigue/Raw Data/S17_2022-06-13.xdf',
 'C:/Users/bribeiroriccioppo/Desktop/mental-fatigue/Raw Data/S18_2022-06-15.xdf',
 'C:/Users/bribeiroriccioppo/Desktop/mental-fatigue/Raw Data/S19_2022-06-15.xdf',
 'C:/Users/bribeiroriccioppo/Desktop/mental-fatigue/Raw Data/S6_2022-05-20.xdf',
 'C:/Users/bribeiroriccioppo/Desktop/mental-fatigue/Raw Data/S7_2022-05-19.xdf',
 'C:/Users/bribeir

In [4]:
participants = data.preprocessing(xdf_files)

Stream 5: Calculated effective sampling rate 94.9479 Hz is different from specified rate 60.0000 Hz.
Stream 5: Calculated effective sampling rate 95.2967 Hz is different from specified rate 60.0000 Hz.
Stream 4: Calculated effective sampling rate 94.8056 Hz is different from specified rate 60.0000 Hz.
Stream 7: Calculated effective sampling rate 94.8889 Hz is different from specified rate 60.0000 Hz.
Stream 3: Calculated effective sampling rate 95.0246 Hz is different from specified rate 60.0000 Hz.
Stream 5: Calculated effective sampling rate 95.0075 Hz is different from specified rate 60.0000 Hz.
Stream 6: Calculated effective sampling rate 94.8952 Hz is different from specified rate 60.0000 Hz.
Stream 1: Calculated effective sampling rate 95.0526 Hz is different from specified rate 60.0000 Hz.
Stream 7: Calculated effective sampling rate 94.8175 Hz is different from specified rate 60.0000 Hz.
Stream 3: Calculated effective sampling rate 94.9544 Hz is different from specified rate 60

## Data

In [5]:
participants_list = ['S6_2022-05-20', 'S7_2022-05-19', 'S8_2022-05-20', 'S9_2022-05-23','S10_2022-05-24',
                     'S11_2022-05-30', 'S12_2022-05-31', 'S13_2022-06-01', 'S14_2022-06-03','S15_2022-06-07',
                     'S16_2022-06-08', 'S17_2022-06-13', 'S18_2022-06-15', 'S19_2022-06-15']

In [79]:
participant = participants_list[13]
blocks_df = participants[participant]['Blocks']
math_tasks_df = participants[participant]['MathTasks']
tone_tasks_df = participants[participant]['ToneTasks']
keyboard_df = participants[participant]['Keyboard']
quest_df = participants[participant]['Questionnaire']

In [80]:
participant

'S19_2022-06-15'

### Blocks

In [81]:
init_time = blocks_df['lsl_init_ts'][0]
blocks_df['lsl_end_ts'] = blocks_df['lsl_end_ts']-init_time
blocks_df['lsl_init_ts'] = blocks_df['lsl_init_ts']-init_time

In [82]:
blocks_df

,block_idx,block_id,lsl_end_ts,lsl_init_ts
0,0.0,0.0,1.407726,0.000000
1,1.0,1.0,121.496874,1.407286
2,2.0,2.0,124.013748,121.496838
3,3.0,3.0,244.934141,124.005239
4,4.0,4.0,248.890359,244.933765
5,5.0,5.0,249.508829,248.889790
6,6.0,6.0,279.571952,249.506595
7,7.0,7.0,280.885722,279.572232
8,8.0,8.0,310.983175,280.883443
9,9.0,9.0,311.330466,310.982892


## Plots for a single participant

### Math task

In [83]:
participant

'S19_2022-06-15'

In [84]:
math_tasks_df['lsl_end_ts'] = math_tasks_df['lsl_end_ts']-init_time
math_tasks_df['lsl_init_ts'] = math_tasks_df['lsl_init_ts']-init_time

In [85]:
math_tasks_df

,block_idx,task_idx,correct,lsl_end_ts,lsl_init_ts
0,6.0,2.0,0.0,253.536849,249.509606
1,6.0,3.0,1.0,258.723021,253.536520
2,6.0,4.0,1.0,260.500853,258.723319
3,6.0,5.0,1.0,262.559460,260.500762
4,6.0,6.0,1.0,263.718356,262.559513
...,...,...,...,...,...
1183,35.0,1200.0,1.0,4339.659726,4336.206411
1184,35.0,1201.0,1.0,4341.796930,4339.659650
1185,35.0,1202.0,1.0,4343.147099,4341.797563
1186,35.0,1203.0,1.0,4345.250354,4343.147041


In [86]:
end_ts_df = math_tasks_df['lsl_end_ts']
init_ts_df = math_tasks_df['lsl_init_ts']

# Reaction times:
rt_math = end_ts_df - init_ts_df

# Answers:
answers = math_tasks_df['correct']

In [87]:
math_tasks_df.insert(5, "rt", rt_math, True)

In [88]:
math_tasks_df

,block_idx,task_idx,correct,lsl_end_ts,lsl_init_ts,rt
0,6.0,2.0,0.0,253.536849,249.509606,4.027242
1,6.0,3.0,1.0,258.723021,253.536520,5.186502
2,6.0,4.0,1.0,260.500853,258.723319,1.777533
3,6.0,5.0,1.0,262.559460,260.500762,2.058698
4,6.0,6.0,1.0,263.718356,262.559513,1.158843
...,...,...,...,...,...,...
1183,35.0,1200.0,1.0,4339.659726,4336.206411,3.453316
1184,35.0,1201.0,1.0,4341.796930,4339.659650,2.137280
1185,35.0,1202.0,1.0,4343.147099,4341.797563,1.349536
1186,35.0,1203.0,1.0,4345.250354,4343.147041,2.103313


In [89]:
false_math_idx = math_tasks_df.index[math_tasks_df['correct'] == 0.0].tolist()
# Linear regression
x_fit = []
for i in range(len(rt_math)):
    x_fit.append(i)

slope_math, intercept_math = np.polyfit(x_fit, rt_math, deg=1) # régression linéaire pour trouver la pente
# --> regarder si c'est significatif ou pas, pcq si c'est pas le cas, on peut dire que la différence n'est pas important
# mais qu'on voit voit quand mm qu'à la fin ya des cas avec des grosses amplitudes
x = range(len(rt_math))
y = slope_math*x+intercept_math
y_zeros = np.zeros(len(false_math_idx))

plt.figure(figsize=(12, 6))
plt.xlabel("Tasks")
plt.ylabel("Reaction Time [s]")
plt.plot(x, rt_math,c='b') # reaction times
#plt.plot(x,rt_math,'-o', markevery=false_math_idx,c='b')
#plt.scatter(x,answers,c='g',label="Answers: 1 = correct, 0 = false, -1 = no answer")
plt.plot(x,y,label='Slope = '+str(round(slope_math,4)),c='g') # linear regression
plt.scatter(false_math_idx, y_zeros, label='False responses', c='r') # false responses
plt.legend(loc='upper right')
plt.savefig('RT_curve_S18.png', dpi=300, bbox_inches='tight')

In [90]:
task_blocks = [13.0, 15.0, 17.0, 19.0, 21.0, 23.0, 25.0, 27.0, 29.0, 31.0, 33.0, 35.0]
fig, ax = plt.subplots(figsize=(12,4))

for idx, block_idx in enumerate(task_blocks):
    sns.boxplot(x='block_idx', y='rt', data=math_tasks_df[math_tasks_df.block_idx.isin(task_blocks)].astype({"block_idx": int}), ax=ax)
    ax.set_xlabel("Blocks")
    ax.set_ylabel("RT [s]");

In [91]:
reduced_math_tasks_df = math_tasks_df.loc[(math_tasks_df['block_idx'] > 12.0)]
ax = sns.countplot(x='block_idx', hue='correct', data=reduced_math_tasks_df.astype({"correct": int, "block_idx": int}))
ax.set_xlabel("Block Idx")
ax.set_ylabel("Count")
legend = ax.get_legend()
legend.set_title("Answers")
new_labels = ["Missed", "Wrong", "Right"]
for t, l in zip(legend.texts, new_labels):
    t.set_text(l);

### Tone task

In [92]:
participant

'S19_2022-06-15'

In [93]:
tone_tasks_df['stimulus_ts'] = tone_tasks_df['stimulus_ts']-init_time
tone_tasks_df['space_ts'] = tone_tasks_df['space_ts']-init_time

In [94]:
tone_tasks_df = participants[participant]['ToneTasks']
response_tone = []

for idx, row in tone_tasks_df.iterrows():
    if ((row["stimulus_id"] == 0.0) and (np.isnan(row["space_ts"]))):
        response_tone.append('Missed')
    elif ((row["stimulus_id"] == 1.0) and not np.isnan(row["space_ts"])):
        response_tone.append('False Alarm')
    else:
        response_tone.append('Correct')
        
tone_tasks_df.insert(6, "response", response_tone, True)

In [95]:
tone_tasks_df.head(20)

,block_idx,task_idx,stimulus_ts,stimulus_id,space_ts,rt,response
0,8.0,15.0,282.620908,1.0,NaN,NaN,Correct
1,8.0,15.0,284.354854,1.0,NaN,NaN,Correct
2,8.0,15.0,286.090211,1.0,NaN,NaN,Correct
3,8.0,15.0,287.824989,1.0,NaN,NaN,Correct
4,8.0,15.0,289.558958,1.0,NaN,NaN,Correct
5,8.0,15.0,291.293476,1.0,NaN,NaN,Correct
6,8.0,15.0,293.030323,1.0,NaN,NaN,Correct
7,8.0,15.0,294.765546,1.0,NaN,NaN,Correct
8,8.0,15.0,296.503838,0.0,297.327164,0.823326,Correct
9,8.0,15.0,298.246843,1.0,NaN,NaN,Correct


In [96]:
high_tone_tasks_df = tone_tasks_df.loc[tone_tasks_df['stimulus_id']==0.0]

In [97]:
high_tone_tasks_df

,block_idx,task_idx,stimulus_ts,stimulus_id,space_ts,rt,response
8,8.0,15.0,296.503838,0.0,297.327164,0.823326,Correct
11,8.0,15.0,301.717127,0.0,302.391879,0.674752,Correct
15,8.0,15.0,308.723020,0.0,309.350332,0.627312,Correct
16,8.0,15.0,310.472520,0.0,311.228815,0.756296,Correct
17,8.0,15.0,310.983157,0.0,311.228815,0.245659,Correct
...,...,...,...,...,...,...,...
2081,35.0,1102.0,4314.188710,0.0,4314.964453,0.775743,Correct
2088,35.0,1102.0,4326.431604,0.0,4327.194237,0.762633,Correct
2096,35.0,1102.0,4340.477798,0.0,4341.337039,0.859241,Correct
2098,35.0,1102.0,4343.965303,0.0,4344.820471,0.855168,Correct


In [98]:
#high_tone_tasks_df.reset_index()

In [99]:
# participants_list = ['S6_2022-05-20', 'S7_2022-05-19', 'S8_2022-05-20', 'S9_2022-05-23','S10_2022-05-24',
#                      'S11_2022-05-30','S12_2022-05-31','S13_2022-06-01', 'S14_2022-06-03']

# for name_id in range(len(participants_list)):
#     name =  participants_list[name_id]
#     tone_tasks_df = participants[name]['ToneTasks']
#     for idx, row in tone_tasks_df.iterrows():
#         if ((row["stimulus_id"] == 0.0) and (np.isnan(row["space_ts"]))):
#             print(name)
#             print(idx)
#             print('Missed')
            
# for name_id in range(len(participants_list)):
#     name =  participants_list[name_id]
#     tone_tasks_df = participants[name]['ToneTasks']
#     for idx, row in tone_tasks_df.iterrows():
#         if ((row["stimulus_id"] == 1.0) and not np.isnan(row["space_ts"])):
#             print(name)
#             print(idx)
#             print('False Alarm')
            

In [100]:
# Plot RTs

#rt_tone = tone_tasks_df['rt']
rt_tone = high_tone_tasks_df['rt']
rt_tone = rt_tone.dropna()

# Linear regression
x_fit = []
for i in range(len(rt_tone)):
    x_fit.append(i)

slope_tone, intercept_tone = np.polyfit(x_fit, rt_tone, deg=1) # régression linéaire pour trouver la pente
x = range(len(rt_tone))
y = slope_tone*x+intercept_tone

plt.figure(figsize=(12, 6))
plt.xlabel("Tone Idx")
plt.ylabel("Reaction Time [s]")
plt.plot(x,rt_tone)
plt.plot(x, y, label='Slope = '+str(round(slope_tone,4)))

plt.legend(loc='upper right')
plt.show()


In [101]:
task_blocks = [13.0, 15.0, 17.0, 19.0, 21.0, 23.0, 25.0, 27.0, 29.0, 31.0, 33.0, 35.0]
fig, ax = plt.subplots(figsize=(12,4))

for idx, block_idx in enumerate(task_blocks):
    sns.boxplot(x='block_idx', y='rt', data=high_tone_tasks_df[high_tone_tasks_df.block_idx.isin(task_blocks)].astype({"block_idx": int}), ax=ax)
    ax.set_xlabel("Blocks")
    ax.set_ylabel("RT [s]");

In [78]:
reduced_high_tone_tasks_df = high_tone_tasks_df.loc[(high_tone_tasks_df['block_idx'] > 12.0)]

ax = sns.countplot(x='block_idx', hue='response', data=reduced_high_tone_tasks_df.astype({"block_idx": int}))

ax.set_xlabel("Block Idx")
ax.set_ylabel("Count")
legend = ax.get_legend()
legend.set_title("Answers")
new_labels = ["Correct", "Miss"]
for t, l in zip(legend.texts, new_labels):
    t.set_text(l);

### Keyboard

In [65]:
keyboard_df['reaction_ts'] = keyboard_df['reaction_ts'] - init_time
print(keyboard_df)

            events  reaction_ts
2    SPACE pressed   525.554571
4    SPACE pressed   527.513502
6    SPACE pressed   538.065236
8    SPACE pressed   547.519416
10   SPACE pressed   560.805381
..             ...          ...
812  SPACE pressed  4602.970637
814  SPACE pressed  4612.151423
816  SPACE pressed  4622.125660
818  SPACE pressed  4627.560304
820  SPACE pressed  4629.048618

[410 rows x 2 columns]
